In [1]:
import pandas as pd
import numpy as np
import warnings
import boto3
import re
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [2]:
client = boto3.client('s3')

In [3]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [4]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/latest_data/feature_prep/')

In [30]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if ('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [31]:
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10min_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_120min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_120min_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_150D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_b

In [32]:
def calculate_macd(df, signal=14, ema_columns=[]):
    for i in range(len(ema_columns)):
        for j in range(i + 1, len(ema_columns)):
            fast_ema = ema_columns[i]
            slow_ema = ema_columns[j]

            # Calculate MACD
            # macd_col_name = f'MACD_{fast_ema}_{slow_ema}'
            # df[macd_col_name] = df[fast_ema] - df[slow_ema]

            # Calculate Signal line
            signal_col_name = f'Signal_{fast_ema}_{slow_ema}_signal{signal}'
            # df[signal_col_name] = df[macd_col_name].ewm(span=signal, adjust=False).mean()
            df[signal_col_name] = (df[fast_ema] - df[slow_ema]).ewm(span=signal, adjust=False).mean()
            

            # # Calculate Histogram
            # histogram_col_name = f'Histogram_{fast_ema}_{slow_ema}_signal{signal}'
            # df[histogram_col_name] = df[macd_col_name] - df[signal_col_name]
    return df


def read_and_calculate_macd(path, signals):
    print(f'Reading from {path}')
    df = pd.read_parquet(path)
    fields = [z for z in df.columns if 'close_ema' in z or 'close_diff_ema' in z]
    df = df[['symbol'] + fields]
    grouped = df.groupby('symbol')
    results = []

    for symbol, group in grouped:
        ema_columns = [z for z in group.columns if z!='symbol']
        ema_columns = sorted(ema_columns, key=lambda x: int(re.search(r'\d+', x).group()))
        for signal in signals:
            print(f'for signal = {signal}')
            group = calculate_macd(group, signal, ema_columns)
        results.append(group)

    # Concatenate all the grouped results back into a single DataFrame
    df = pd.concat(results)
    loc = path.replace('.parquet', '_macd.parquet')
    print(f'Saving to {loc}')
    df.to_parquet(loc)
    del df, group, results


for path in tqdm(paths):
    base_time_interval = path.split('/')[-1].split('.')[0].split('_')[-2]
    if 'min' in base_time_interval:
        if int(base_time_interval.replace('min', ''))<=10:
            signal = [11, 13, 17, 20, 26]
        elif int(base_time_interval.replace('min', ''))<=30:
            signal = [11, 13, 17, 20, 26]
        else:
            signal = [11, 13, 17, 20, 26, 30, 50]
    elif 'D' in base_time_interval:
        if int(base_time_interval.replace('D', '')) < 100:
            signal = [11, 13, 17, 20, 26]
        else: 
            signal = [11, 13, 17, 20, 26, 30, 50]
    read_and_calculate_macd(path, signal)

  0%|          | 0/50 [00:00<?, ?it/s]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base_macd.parquet


  2%|▏         | 1/50 [00:01<01:07,  1.37s/it]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff_macd.parquet


  4%|▍         | 2/50 [00:02<01:01,  1.28s/it]

...

 96%|█████████▌| 48/50 [05:30<00:22, 11.05s/it]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base.parquet
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base_macd.parquet


 98%|█████████▊| 49/50 [05:33<00:08,  8.64s/it]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff.parquet
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
for signal = 11
for signal = 13
for signal = 17
for signal = 20
for signal = 26
for signal = 30
for signal = 50
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff_macd.parquet


100%|██████████| 50/50 [05:37<00:00,  6.76s/it]


### eda

In [ ]:
# y = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet")
# y = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet")

In [2]:
x = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base_macd.parquet")
x.loc['2024-11-19 16:50:00']
x.shape # (1160890, 149)
x = x.reset_index()[['us_eastern_timestamp', 'symbol']]

In [3]:
y = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_macd.parquet")
y.loc['2024-11-19 16:50:00']
y.shape # (392417, 149)
y = y.reset_index()[['us_eastern_timestamp', 'symbol']]

In [4]:
x[x.symbol=='SPY']

,us_eastern_timestamp,symbol
696460,2024-01-02 04:00:00,SPY
696461,2024-01-02 04:02:00,SPY
696462,2024-01-02 04:04:00,SPY
696463,2024-01-02 04:06:00,SPY
696464,2024-01-02 04:08:00,SPY
...,...,...
928775,2024-11-19 19:50:00,SPY
928776,2024-11-19 19:52:00,SPY
928777,2024-11-19 19:54:00,SPY
928778,2024-11-19 19:56:00,SPY


In [5]:
y[y.symbol=='SPY']

,us_eastern_timestamp,symbol
198636,2024-01-02 04:00:00,SPY
198637,2024-01-02 04:01:00,SPY
198638,2024-01-02 04:02:00,SPY
198639,2024-01-02 04:03:00,SPY
198640,2024-01-02 04:05:00,SPY
...,...,...
374553,2024-11-19 19:54:00,SPY
374554,2024-11-19 19:56:00,SPY
374555,2024-11-19 19:57:00,SPY
374556,2024-11-19 19:58:00,SPY


In [ ]:
del x
# 175922 232320 

In [8]:
# y[(y.symbol=='SPY')&(~y.us_eastern_timestamp.isin(x[x.symbol=='SPY'].us_eastern_timestamp))]
x[(x.symbol=='SPY')&(~x.us_eastern_timestamp.isin(y[y.symbol=='SPY'].us_eastern_timestamp))]

,us_eastern_timestamp,symbol
696462,2024-01-02 04:04:00,SPY
696467,2024-01-02 04:14:00,SPY
696469,2024-01-02 04:18:00,SPY
696470,2024-01-02 04:20:00,SPY
696471,2024-01-02 04:22:00,SPY
...,...,...
928766,2024-11-19 19:32:00,SPY
928767,2024-11-19 19:34:00,SPY
928768,2024-11-19 19:36:00,SPY
928769,2024-11-19 19:38:00,SPY


In [10]:
y[y.symbol=='SPY'].tail(30)

,us_eastern_timestamp,symbol
374528,2024-11-19 18:26:00,SPY
374529,2024-11-19 18:27:00,SPY
374530,2024-11-19 18:28:00,SPY
374531,2024-11-19 18:30:00,SPY
374532,2024-11-19 18:31:00,SPY
374533,2024-11-19 18:33:00,SPY
374534,2024-11-19 18:34:00,SPY
374535,2024-11-19 18:37:00,SPY
374536,2024-11-19 18:39:00,SPY
374537,2024-11-19 18:40:00,SPY
